In [191]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
import math
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from wordcloud import WordCloud
from textwrap import wrap
from textblob import TextBlob
import textstat
import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords 
custom_stopwords=stopwords.words('english')
# custom_stopwords.extend(['said','-','like','many','told','can','could','would','should','a','want','will','about', 'actually', 'almost', 'also', 'although', 'always', 'am', 'an', 'and', 'any', 'are', 'as', 'at', 'be', 'became', 'become', 'but', 'by', 'can', 'could', 'did', 'do', 'does', 'each', 'either', 'else', 'for', 'from', 'had', 'has', 'have', 'hence', 'how', 'i', 'if', 'in', 'is', 'it', 'its', 'just', 'maybe', 'me', 'might', 'mine', 'must', 'my', 'mine', 'must', 'my', 'neither', 'nor', 'not', 'of', 'oh', 'ok', 'when', 'where', 'whereas', 'wherever', 'whenever', 'whether', 'which', 'while', 'who', 'whom', 'whoever', 'whose', 'why', 'will', 'with', 'within', 'without', 'would', 'yes', 'yet', 'you', 'your'])

custom_stopwords.extend(['said','also','need','the','year','take','took','got','years','day','days','used','use','month','mr','mrs','ms','since','hence','henceforth','months','say','says','get','time','come','much','know','u','-','like','many','told','can','could','would','should','a','want','will','about', 'actually', 'almost', 'also', 'although', 'always', 'am', 'an', 'and', 'any', 'are', 'as', 'at', 'be', 'became', 'become', 'but', 'by', 'can', 'could', 'did', 'do', 'does', 'each', 'either', 'else', 'for', 'from', 'had', 'has', 'have', 'hence', 'how', 'i', 'if', 'in', 'is', 'it', 'its', 'just', 'may', 'maybe', 'me', 'might', 'mine', 'must', 'my', 'mine', 'must', 'my', 'neither', 'nor', 'not', 'of', 'oh', 'ok', 'when', 'where', 'whereas', 'wherever', 'whenever', 'whether', 'which', 'while', 'who', 'whom', 'whoever', 'whose', 'why', 'will', 'with', 'within', 'without', 'would', 'yes', 'yet', 'you', 'your'])

In [192]:
# Load CSV file into DataFrame
df = pd.read_excel('output-merged.xlsx')
df.head(20)

In [193]:
df.shape
text = df[['Title', 'Content', 'Number of Words', 'Category','Link']]
text.head()

In [194]:
df = df[['Title', 'Content', 'Number of Words']]
df

## GROUPING CATEGORIES

In [195]:
sport_categories = ["football", "tennis", "sport", "motorsport", "golf","cricket","hockey"]
country_categories =["us","uk","europe","australia","americas","africa","asia","china","india","middleeast","weather"]
tech=["tech","technology"]
celeb=["media","celebrities","entertainment"]
business=["business","Business","homes","success","cars"]
health=["health","disability","newsbeat"]
edu=["education","explainers"]
pol=["politics","opinions"]
travel=["NaN"]
science=["science"]
money=["economy","investing"]

# Define a dictionary to map original category values to subcategory values
subcategories_dict = {"science":"science","NaN":"travel","education":"education","explainers":"explainers","cricket":"cricket","tech":"tech","technology":"technology","business":"business","Business":"Business","health":"health","disability":"disability","newsbeat":"newsbeat","football": "football", "tennis": "tennis", "sport": "sport", "motorsport": "motorsport", "golf": "golf","us":"us","uk":"uk","europe":"europe","australia":"australia","americas":"americas","africa":"africa","asia":"asia","china":"china","india":"india","middleeast":"middleast","hockey":"hockey","cricket":"cricket","economy":"economy","investing":"investing","homes":"homes","opinions":"opinions","politics":"politics","success":"success","cars":"cars","science":"science","weather":"weather"}
#print(df['Category'])
# Update the "Category" column and create a new "SubCategory" column
text.loc[text['Category'].isin(sport_categories), 'SubCategory'] = text['Category']
text.loc[text['Category'].isin(sport_categories), 'Category'] = 'SPORTS'
text.loc[text['Category'].isin(country_categories), 'SubCategory'] = text['Category']
text.loc[text['Category'].isin(country_categories), 'Category'] = 'WORLD NEWS'
text.loc[text['Category'].isin(tech), 'SubCategory'] = text['Category']
text.loc[text['Category'].isin(tech), 'Category'] = 'TECH'
text.loc[text['Category'].isin(celeb), 'SubCategory'] = text['Category']
text.loc[text['Category'].isin(celeb), 'Category'] = 'ENTERTAINMENT'
text.loc[text['Category'].isin(business), 'SubCategory'] = text['Category']
text.loc[text['Category'].isin(business), 'Category'] = 'BUSINESS'
text.loc[text['Category'].isin(health), 'SubCategory'] = text['Category']
text.loc[text['Category'].isin(health), 'Category'] = 'HEALTHY LIVING'
text.loc[text['Category'].isin(edu), 'SubCategory'] = text['Category']
text.loc[text['Category'].isin(edu), 'Category'] = 'EDUCATION'
text.loc[text['Category'].isin(pol), 'SubCategory'] = text['Category']
text.loc[text['Category'].isin(pol), 'Category'] = 'POLITICS'
text.loc[text['Category'].isin(travel), 'SubCategory'] = text['Category']
text.loc[text['Category'].isin(travel), 'Category'] = 'TRAVEL'
text.loc[text['Category'].isin(science), 'SubCategory'] = text['Category']
text.loc[text['Category'].isin(science), 'Category'] = 'SCIENCE'
text.loc[text['Category'].isin(money), 'SubCategory'] = text['Category']
text.loc[text['Category'].isin(money), 'Category'] = 'MONEY'

#print(df['Category'].isin(sport_categories))
# Map the subcategory values using the subcategories_dict dictionary
text['SubCategory'] = text['SubCategory'].map(subcategories_dict)

# Save the updated DataFrame to a CSV file
text.to_csv('output-merge.csv', index=False)

 # Define the desired categories
desired_categories = ['SPORTS', 'WORLD NEWS', 'TECH', 'ENTERTAINMENT', 'BUSINESS', 'HEALTHY LIVING', 'EDUCATION', 'POLITICS', 'TRAVEL', 'SCIENCE', 'MONEY']

# Create a new DataFrame with only the desired categories
text_filtered = text[text['Category'].isin(desired_categories)]

# Save the filtered DataFrame to a CSV file
text_filtered.to_csv('output-filtered.csv', index=False)

text.head()

## Replace empty "Content" with corresponding "Title"

In [196]:
import numpy as np
text['Content'] = text['Content'].replace(' ', np.nan)
text['Content'].fillna(text['Title'], inplace=True)
text

## Word count for the "Content" column

In [197]:
def count_words(cell):
    words = cell.split()
    return len(words)

# Apply the count_words function to the 'Content' column using apply() and lambda
text['Word Count(new)'] = text['Content'].apply(lambda x: count_words(x))
text

## Data Preprocessing
There is no missing data, therefore, we can move to the next stage. For Term frequency analysis, it is essential that the text data be preprocessed.

* Lowercase
* Remove punctutations
* Stopword removal

In [198]:
def clean(content):
    
    content = content.lower()
    content = re.sub('[^a-z A-Z 0-9-]+', '', content)
    content = " ".join([word for word in content.split() if word not in custom_stopwords])
    content = content.replace("\n", "").strip()
    return content

def tclean(title):
    
    title = title.replace("\n", "").strip()
    return title

text['Content'] = text['Content'].apply(clean)
text['Title'] = text['Title'].apply(clean)
text

In [199]:
text['Word Count(new)'] = text['Content'].apply(lambda x: count_words(x))
text

In [ ]:
def corpus(text):
    text_list = text.split()
    return text_list

text['Content_list'] = text['Content'].apply(corpus)
df['Content_list']=df['Content'].apply(corpus)

text.to_csv('cleaned.csv')
text

## Word frequency 

Word frequency analysis is a technique used to determine the frequency of each word in a given text or corpus. 

In Python, the Natural Language Toolkit (NLTK) library provides a range of tools for text analysis, including word frequency analysis. 

NLTK's word frequency analysis capabilities involve counting the occurrence of each word in a given text and then generating a frequency distribution table that lists each word and its frequency in descending order. 

NLTK's frequency distribution table can be used to identify the most common words in a text, which can provide insights into the topic or theme of the text. 

NLTK's word frequency analysis is a useful tool for a variety of applications, such as content analysis, topic modeling, and keyword research. 

Additionally, NLTK provides the ability to customize the word frequency analysis by removing stopwords, punctuation, and other noise words to obtain a more accurate representation of the frequency of meaningful words in a given text.

In [200]:
all_words = ''.join([word for word in df['Content']]).replace('\n', ' ')

In [201]:
# Create a WordCloud object
wordcloud = WordCloud(width=1000, height=700, max_font_size=110, background_color="black",colormap='rainbow', max_words=1000, contour_width=3, contour_color='red',collocations=False,stopwords=custom_stopwords)

# Generate a word cloud
wordcloud.generate(all_words)

# Visualize the word cloud
wordcloud.to_image()

In [202]:
from collections import Counter
from tqdm import trange

corpus = []
for i in trange(text1.shape[0], ncols=150, nrows=10, colour='green', smoothing=0.8):
    corpus += text['Content_list'][i]
len(corpus)

mostCommon = Counter(corpus).most_common(15)
mostCommon

In [ ]:
words = []
freq = []
for word, count in mostCommon:
    words.append(word)
    freq.append(count)

In [ ]:
import seaborn as sns
sns.barplot(x=freq, y=words)
plt.title('Top 15 Most Frequently Occuring Words')
plt.show()

> A stat that shows most news is "he said, she said"! XD

## Most Frequently occuring N_grams

An n-gram is sequence of n words in a text. Most words by themselves may not present the entire context. Typically adverbs such as 'most' or 'very' are used to modify verbs and adjectives. Therefore, n-grams help analyse phrases and not just words which can lead to better insights.

A Bi-gram means two words in a sequence. 'Very good' or 'Too great'
A Tri-gram means three words in a sequence. 'How was your day' would be broken down to 'How was your' and 'was your day'.

For separating text into n-grams, we will use *CountVectorizer* from Sklearn.

<h2><b>BIGRAMS</b></h2>

In [ ]:
cv = CountVectorizer(ngram_range=(2,2))
bigrams = cv.fit_transform(text['Content'])


In [203]:
count_values = bigrams.toarray().sum(axis=0)
ngram_freq = pd.DataFrame(sorted([(count_values[i], k) for k, i in cv.vocabulary_.items()], reverse = True))
ngram_freq.columns = ["frequency", "bi-gram"]

In [ ]:
sns.barplot(x=ngram_freq['frequency'][:15], y=ngram_freq['bi-gram'][:15])
plt.title('Top 10 Most Frequently Occuring Bigrams')
plt.show()

<h2><b>TRIGRAMS</b></h2>

In [ ]:
cv1 = CountVectorizer(ngram_range=(3,3))
trigrams = cv1.fit_transform(text['Content'])
count_values = trigrams.toarray().sum(axis=0)
ngram_freq = pd.DataFrame(sorted([(count_values[i], k) for k, i in cv1.vocabulary_.items()], reverse = True))
ngram_freq.columns = ["frequency", "Tri-gram"]

In [ ]:
sns.barplot(x=ngram_freq['frequency'][:15], y=ngram_freq['Tri-gram'][:15])
plt.title('Top 10 Most Frequently Occuring Trigrams')
plt.show()

First, We’ll take a look at the number of characters present in each sentence. This can give us a rough idea about the news headline length.

In [ ]:
text['Title'].str.len().hist()

Each headline has about 100 characters!

In [ ]:
text['Title'].str.split().map(lambda x: len(x)).hist()

Each headline has about 8 words!

<h1><b>Latest News per day per country Visualization</b></h1>

In [ ]:
import pycountry

In [ ]:
# Create a set of all country names
countries = {country.name for country in pycountry.countries}
# Check for country names in the text
for text in text['Title']:
    for word in text.split():
        if word in countries:
            print (word)

<h2><b>Chloropleth Maps</h2><b>

In [ ]:
import plotly.express as px
import pycountry
import pandas as pd


In [ ]:
import pycountry
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

# Create a set of all country names
countries = {country.name for country in pycountry.countries}
# Count the number of occurrences of each country in the 'Title' column
counts = text['Title'].apply(lambda x: pd.Series([word for word in x.split() if word in countries]))\
                    .stack()\
                    .reset_index(drop=True)\
                    .value_counts()\
                    .reset_index()\
                    .rename(columns={'index': 'country', 0: 'count'})
print(counts)
# Get the alpha-3 codes for each country
def get_alpha3(country_name):
    try:
        for country in pycountry.countries:
           return pycountry.countries.search_fuzzy(country_name)[0].alpha_3
    except:
        return None
counts['alpha_3'] = counts['country'].apply(get_alpha3)

# Read the world map shapefile
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world['alpha_3'] = world['iso_a3']
world['count'] = counts['count']

# Merge the world map with the counts DataFrame
merged = world.merge(counts[['alpha_3', 'country']], on='alpha_3', how='left')

# # Generate the chloropleth map
# fig, ax = plt.subplots(figsize=(12, 8))
# merged.plot(column='name', cmap='Blues', legend=False, ax=ax, edgecolor='grey', linewidth=0.5, k=5, figsize=(10, 6))
# ax.set_title('Country Counts')
# ax.set_axis_off()
# plt.show()
merged.to_csv('country.csv')
counts.to_csv('count.csv')

<h2><b>More Detailed visualization using Chloropleth Maps</b></h2>

<h3>used bokeh library to do visualization of the news counts per country</h3>

In [ ]:
import pycountry
import pandas as pd
import geopandas as gpd
from bokeh.io import show
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, HoverTool
from bokeh.palettes import Blues,Plasma,Purples,Blues256
from bokeh.models import LinearColorMapper
from bokeh.palettes import Magma,Viridis,Inferno,Cividis,Spectral,Category10,Category20,Pastel2

# Create a set of all country names
countries = {country.name for country in pycountry.countries}
# print(countries)
# Count the number of occurrences of each country in the 'Title' column
counts = text['Title'].apply(lambda x: pd.Series([word for word in x.split() if word in countries]))\
                    .stack()\
                    .reset_index(drop=True)\
                    .value_counts()\
                    .reset_index()\
                    .rename(columns={'index': 'country', 0: 'count'})



# Get the alpha-3 codes for each country
def get_alpha3(country_name):
    if country_name == 'US' or country_name == 'USA' or country_name == 'America' or country_name=='United States':
        return pycountry.countries('U.S.')
    try:
        for country in pycountry.countries:
           return pycountry.countries.search_fuzzy(country_name)[0].alpha_3
    except:
        return None
counts['alpha_3'] = counts['country'].apply(get_alpha3)

# Read the world map shapefile
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world['alpha_3'] = world['iso_a3']

# Merge the world map with the counts DataFrame
merged = world.merge(counts[['alpha_3', 'country', 'count']], on='alpha_3', how='left')

# Convert merged DataFrame to GeoJSONDataSource
geosource = GeoJSONDataSource(geojson=merged.to_json())
low = max(counts['count'])
high = min(counts['count'])
# Define color palette
palette = Blues256[::-1]
# Instantiate the figure object
# p = figure(title='Country Counts', plot_height=600, plot_width=950, toolbar_location=None, tools=[])
p = figure(plot_height=840, plot_width=1500, toolbar_location=None, tools=[],background='white',
           background_fill_color='black')


# Add the polygons to the map
p.patches('xs', 'ys', source=geosource, fill_color={'field': 'count', 'transform': LinearColorMapper(palette=palette)}, line_color='white', line_width=0.5, fill_alpha=1)

# Add the hover tool
hover = HoverTool(tooltips=[('Country', '@name'), ('Count', '@count')], mode='mouse')
p.add_tools(hover)

# Set the map properties
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
p.axis.visible = False
p.outline_line_color = None

# Show the map
show(p)